# **TAREA LanceDB**
- Considera usar ANN para cada búsqueda o filtro

**Task 1: Consulta avanzada con proyección y filtro**

Instrucciones:
1. Genera un vector aleatorio con la misma dirección que los embeddings que están en la tabla mis_vectores
2. Realiza una búsqueda en la tabla para encontrar los 5 elementos más cercanos
3. Proyecta los resultados para mostrar solo las columnas item y _distance
4. Excluye de los resultado los elementos cuyo nombre sea 'item 500'

Pregunta: ¿Cuáles son los cinco elementos más cercanos que cumplen con los criterios y cuál es la distancia de cada uno?

In [4]:
import lancedb
import numpy as np
import pandas as pd

# Connect to the database
uri = "data/ANN"
db = lancedb.connect(uri)

# Open the existing table
tbl = db.open_table("mis_vectores")

# Generate a random vector with the same dimensionality as the embeddings
# The embeddings are 1536-dimensional based on the previous code
np.random.seed(42)  # For reproducibility
query_vector = np.random.random(1536).astype("float32")

# Perform the similarity search
results = (
    tbl.search(query_vector)  # Search with the random vector
    .limit(5)  # Limit to 5 results
    .where("item != 'item 500'")  # Exclude 'item 500'
    .to_pandas()  # Convert directly to pandas
)

print("Five nearest neighbors:")
print(results)

Five nearest neighbors:
                                              vector         item   _distance
0  [0.37454012, 0.9507143, 0.7319939, 0.5986585, ...       item 0    0.000000
1  [0.20024236, 0.60035306, 0.2857333, 0.39349523...    item 1523  230.041855
2  [0.13669865, 0.3894892, 0.049534895, 0.8630013...   item 38321  230.824173
3  [0.3673092, 0.6906272, 0.82208246, 0.6918987, ...   item 34220  231.273346
4  [0.2601927, 0.72232693, 0.77136034, 0.11831053...  item 140761  231.335159


**Task 2: Creación de tablas**

Instrucciones:
1. Define un nuevo esquema para una tabla vacía con las siguientes columnas:
*   vector (vector de 4 dimensiones)
*   nombre
*   categoria
2. Crea una tabla vacía llamada nueva_tabla usando el esquema
3. Inserta 5 registros en la tabla
4. Muestra el contenido de la tabla





In [6]:
import lancedb
import numpy as np
import pandas as pd
from lancedb.pydantic import LanceModel, Vector

# Define the schema using Pydantic LanceModel
class NuevoEsquema(LanceModel):
    vector: Vector(4)  # Vector de 4 dimensiones
    nombre: str
    categoria: str

# Connect to the database
uri = "data/ANN"
db = lancedb.connect(uri)

# Create an empty table with the defined schema
nueva_tabla = db.create_table(
    "nueva_tabla", 
    schema=NuevoEsquema.to_arrow_schema(), 
    exist_ok=True,  # Permite sobrescribir si ya existe
    mode="overwrite"  # Sobrescribe la tabla si ya existe
)

# Preparar datos para insertar
datos_insertar = [
    {
        "vector": [1.1, 2.2, 3.3, 4.4],
        "nombre": "Elemento A",
        "categoria": "Tipo 1"
    },
    {
        "vector": [0.5, 1.5, 2.5, 3.5],
        "nombre": "Elemento B", 
        "categoria": "Tipo 2"
    },
    {
        "vector": [4.4, 3.3, 2.2, 1.1],
        "nombre": "Elemento C",
        "categoria": "Tipo 1"
    },
    {
        "vector": [2.0, 3.0, 4.0, 5.0],
        "nombre": "Elemento D",
        "categoria": "Tipo 3"
    },
    {
        "vector": [5.5, 4.4, 3.3, 2.2],
        "nombre": "Elemento E",
        "categoria": "Tipo 2"
    }
]

# Insertar los registros
nueva_tabla.add(datos_insertar)

# Mostrar el contenido de la tabla
resultados = nueva_tabla.to_pandas()
print("Contenido de la tabla:")
print(resultados)

Contenido de la tabla:
                 vector      nombre categoria
0  [1.1, 2.2, 3.3, 4.4]  Elemento A    Tipo 1
1  [0.5, 1.5, 2.5, 3.5]  Elemento B    Tipo 2
2  [4.4, 3.3, 2.2, 1.1]  Elemento C    Tipo 1
3  [2.0, 3.0, 4.0, 5.0]  Elemento D    Tipo 3
4  [5.5, 4.4, 3.3, 2.2]  Elemento E    Tipo 2


**Task 3: Actualización de vectores y filtrado**

Instrucciones:
1. Crea una tabla utilizando un DataFrame de Pandas con las siguientes columnas:
- id (Entero).
- vector (Lista de tres números flotantes)
2. Actualiza el vector de la fila donde id=3 a [10.0, 11.0, 10.0]
3. Filtra la tabla para mostrar solo las filas donde al menos un valor del vector sea mayor a 9.0


In [8]:
import lancedb
import pandas as pd
import numpy as np

# Crear un DataFrame de Pandas con los datos iniciales
data = pd.DataFrame({
    'id': [1, 2, 3, 4, 5],
    'vector': [
        [1.0, 2.0, 3.0],
        [4.0, 5.0, 6.0],
        [7.0, 8.0, 9.0],
        [2.0, 3.0, 4.0],
        [5.0, 6.0, 7.0]
    ]
})

# Conectar a la base de datos
uri = "data/ANN"
db = lancedb.connect(uri)

# Crear la tabla en LanceDB
tabla = db.create_table("actualizacion_tabla", data, exist_ok=True, mode="overwrite")

# Actualizar el vector de la fila donde id=3 a [10.0, 11.0, 10.0]
# En LanceDB, necesitamos reemplazar el registro completo
def update_vector(table, id_to_update, new_vector):
    # Obtener todos los registros
    df = table.to_pandas()
    
    # Encontrar el índice de la fila a actualizar
    row_index = df[df['id'] == id_to_update].index[0]
    
    # Crear un nuevo registro con el vector actualizado
    updated_row = df.loc[row_index].to_dict()
    updated_row['vector'] = new_vector
    
    # Eliminar la fila antigua y agregar la nueva
    table.delete(f"id = {id_to_update}")
    table.add([updated_row])

# Realizar la actualización
update_vector(tabla, 3, [10.0, 11.0, 10.0])

# Filtrar para mostrar filas donde al menos un valor del vector sea mayor a 9.0
# Primero, convertir a pandas para facilitar el filtrado
df_filtrado = tabla.to_pandas()
df_filtrado_resultado = df_filtrado[df_filtrado['vector'].apply(lambda x: any(val > 9.0 for val in x))]

print("Tabla después de la actualización y filtrado:")
print(df_filtrado_resultado)

Tabla después de la actualización y filtrado:
   id              vector
4   3  [10.0, 11.0, 10.0]


**Task 4: Embeddings multimodales y búsqueda combinada**

Instrucciones:
1. Crea una tabla con datos de texto e imágenes combinados. Incluye las siguientes columnas:
- texto (Texto).
- imagen (Nombre del archivo de imagen).
- embedding_texto (Vector del texto generado con SentenceTransformer).
- embedding_imagen (Vector de la imagen generado con ResNet18).
2. Realiza una consulta para encontrar los elementos con un texto similar a "La tecnología avanza rápido" y una imagen visualmente similar a un color predominantemente azul.
3. Muestra los resultados combinados ordenados por la menor distancia promedio entre ambos embeddings.


In [12]:
import numpy as np
import pandas as pd
import torch
from torchvision.models import resnet18, ResNet18_Weights
from torchvision import transforms
from PIL import Image
from sentence_transformers import SentenceTransformer
import lancedb
import os

# **Paso 1: Configuración del modelo**
# Modelo para texto
modelo_texto = SentenceTransformer('all-MiniLM-L6-v2')

# Modelo para imágenes
modelo_imagen = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
modelo_imagen.fc = torch.nn.Identity()  # Retirar la capa final para obtener el embedding
modelo_imagen.eval()

# Transformaciones para procesar imágenes
transformacion_imagen = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# **Paso 2: Ruta a la carpeta de imágenes**
directorio_imagenes = "data/images"  # Ajusta la ruta según tu estructura de proyecto

# **Paso 3: Procesar cada imagen y generar embeddings**
embeddings_imagenes = []
nombres_imagenes = []

for archivo_imagen in os.listdir(directorio_imagenes):
    if archivo_imagen.endswith(".jpg") or archivo_imagen.endswith(".png"):
        ruta_imagen = os.path.join(directorio_imagenes, archivo_imagen)
        imagen = Image.open(ruta_imagen).convert("RGB")
        tensor_imagen = transformacion_imagen(imagen).unsqueeze(0)  # Agregar dimensión batch

        # Generar embedding de la imagen
        with torch.no_grad():
            embedding = modelo_imagen(tensor_imagen).squeeze().numpy().astype('float32')

        nombres_imagenes.append(archivo_imagen)  # Guardar nombre de la imagen
        embeddings_imagenes.append(embedding.tolist())  # Guardar embedding como lista

# **Paso 4: Generar embeddings para textos**
textos = [
    "La tecnología avanza rápido",
    "El cielo es azul y hermoso",
    "Los avances científicos son notables",
    "Una escena nocturna tranquila",
    "Un paisaje marino con tonos azules"
]

embeddings_textos = [modelo_texto.encode(texto).tolist() for texto in textos]

# **Paso 5: Crear DataFrame con texto, imágenes y embeddings**
df_multimodal = pd.DataFrame({
    "texto": textos,
    "imagen": nombres_imagenes[:len(textos)],  # Emparejar imágenes con textos disponibles
    "embedding_texto": embeddings_textos,
    "embedding_imagen": embeddings_imagenes[:len(textos)]  # Asegurarse de que coincidan las longitudes
})

print("Embeddings multimodales generados:")
print(df_multimodal.head())

# **Paso 6: Guardar en LanceDB**
# Conexión a LanceDB
bd = lancedb.connect("data/introdb")

# Crear tabla en LanceDB
tabla_multimodal = bd.create_table("multimodal_embeddings", data=df_multimodal, mode="overwrite")
print("Tabla 'multimodal_embeddings' creada con éxito.")

# **Paso 7: Realizar búsqueda combinada**
# Embedding del texto objetivo
consulta_texto = "La tecnología avanza rápido"
embedding_consulta_texto = modelo_texto.encode(consulta_texto)

# Crear un embedding de 512 dimensiones para el color azul
embedding_consulta_imagen = np.zeros(512)
embedding_consulta_imagen[-1] = 1.0  # Predomina el azul

# Realizar búsqueda en la tabla
resultados = tabla_multimodal.to_pandas()

# Calcular distancia promedio entre embeddings
def calcular_distancia_combinada(fila):
    distancia_texto = np.linalg.norm(np.array(fila["embedding_texto"]) - np.array(embedding_consulta_texto))
    distancia_imagen = np.linalg.norm(np.array(fila["embedding_imagen"]) - embedding_consulta_imagen)
    return (distancia_texto + distancia_imagen) / 2

resultados["distancia"] = resultados.apply(calcular_distancia_combinada, axis=1)

# Ordenar por la menor distancia promedio
resultados = resultados.sort_values("distancia").reset_index(drop=True)

# **Paso 8: Mostrar los resultados combinados**
print("Resultados ordenados por menor distancia promedio:")
print(resultados[["texto", "imagen", "distancia"]])


Embeddings multimodales generados:
                                  texto                            imagen  \
0           La tecnología avanza rápido                        ivf_pq.png   
1            El cielo es azul y hermoso                      imagen_6.jpg   
2  Los avances científicos son notables                      imagen_2.jpg   
3         Una escena nocturna tranquila  lancedb_embedded_explanation.png   
4    Un paisaje marino con tonos azules                      imagen_4.jpg   

                                     embedding_texto  \
0  [0.013738852925598621, -0.015867536887526512, ...   
1  [0.002947152592241764, 0.07605063170194626, -0...   
2  [0.02340988628566265, -0.0009007267071865499, ...   
3  [0.05347878113389015, 0.04372989013791084, -0....   
4  [-0.029234131798148155, 0.11961360275745392, -...   

                                    embedding_imagen  
0  [1.3739445209503174, 0.47907373309135437, 0.43...  
1  [0.5759119987487793, 1.5680556297302246, 0.777...  
